In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from pathlib import Path
from Utils.utils import generate_and_save_denoised_images


def main(clean_images_dir, noisy_images_dir, output_dir):
    """
    Main function to run the denoising process on all images in the specified directories.
    
    Args:
        clean_images_dir (str): Path to the directory containing clean images.
        noisy_images_dir (str): Path to the directory containing noisy images.
        output_dir (str): Path to the directory where denoised images will be saved.
    """
    from pathlib import Path
    import cv2
    import numpy as np
    from Utils.utils import LorentzianLoss, SparseGradientPrior, GridMRF, GibbsSampler

    clean_images_path = Path(clean_images_dir)
    noisy_images_path = Path(noisy_images_dir)
    output_path = Path(output_dir)

    output_path.mkdir(parents=True, exist_ok=True)

    rows = []
    for clean_image_path in clean_images_path.glob('*.png'):
        print(f"Processing {clean_image_path}...")
        noisy_image_path = list(noisy_images_path.glob(f"{clean_image_path.stem}*"))[0]
        if not noisy_image_path.exists():
            print(f"Noisy image for {clean_image_path} not found, skipping.")
            continue
        denoised_image_paths = generate_and_save_denoised_images(str(noisy_image_path), str(output_path))
        
        clean_image = cv2.imread(str(clean_image_path), cv2.IMREAD_COLOR)
        clean_image = cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB)
        
        row = {}
        for name, path in (denoised_image_paths | {"noisy": noisy_image_path}).items():
            image = cv2.imread(str(path), cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            l1 = np.mean(np.abs(clean_image - image))
            l2 = np.sqrt(np.sum((clean_image - image) * (clean_image - image)))
            mse = np.mean((clean_image - image) ** 2)
            #psnr
            psnr = cv2.PSNR(clean_image, image)
            row[f"{name}_psnr"] = psnr
            row[f"{name}_l1"] = l1
            row[f"{name}_l2"] = l2
            row[f"{name}_mse"] = mse
            
        rows.append(row)
        
    import pandas as pd
    df = pd.DataFrame(rows)
    df.to_csv(output_path / 'denoising_results.csv', index=False)
        
        
if __name__ == "__main__":
    import argparse

    clean_images_dir = Path("../Data/Raw/examples")
    noisy_images_dir = Path("../Data/Transformed/examples")
    
    main(clean_images_dir, noisy_images_dir, Path("outptut"))
        

Processing ../Data/Raw/examples/parrot.png...
[DEBUG] Starting MAPE with SA + Quadratic
[DEBUG] Starting MAPE with SA + Potts
[DEBUG] Starting MMSE with Quadratic
[DEBUG] Starting MMSE with Potts
✓ Completed: mape_sa_quad
✓ Completed: mmse_quad
✓ Completed: mape_sa_potts
✓ Completed: mmse_potts
Processing ../Data/Raw/examples/lena_color.png...
[DEBUG] Starting MAPE with SA + Quadratic
[DEBUG] Starting MAPE with SA + Potts
[DEBUG] Starting MMSE with Quadratic
[DEBUG] Starting MMSE with Potts
✓ Completed: mape_sa_quad
✓ Completed: mmse_quad
✓ Completed: mape_sa_potts
✓ Completed: mmse_potts
